In [20]:
from IPython.display import clear_output

import panel as pn
import ipywidgets as widgets
import pandas as pd
import altair as alt
import pathlib
import datetime
from vega_datasets import data
import os
import pycountry

TIMESTAMP_START = "First Downloaded At"
TIMESTAMP_END = "Last Downloaded At"
COUNT = "COUNT"
IP_ADDR = "Client IP"
DATA_AMOUNT = "File Size (MB)"
FILE_NAME = "Filename"
URL = "URL"
DOWNLOAD_COUNT = "Download Count"
APP = "Application"
FILE_EXT = "File Extension"


#User Options
DATA_AMOUNT_DSC = "Downloaded data amount"
TIME_PLOT_DSC = "Time plot"


prefix_file_path = os.path.join("Task 2", "536535_data", "536535_data")
download_file_path = os.path.join(prefix_file_path, "m365_exfiltration", "single_user_download.csv")
dir_path = pathlib.Path().resolve()

download_data: pd.DataFrame = pd.DataFrame(pd.read_csv(dir_path / download_file_path))

pn.extension('vega')

style = {'description_width': 'initial'}

top_n_widget = widgets.Dropdown(
    options=['10', '20', '30', '40', '50'],
    value='10',
    description='Top N stats',
    disabled=False,
    style = style,
)

data_type_widget = widgets.ToggleButtons(
    options=[DATA_AMOUNT_DSC, TIME_PLOT_DSC],
    description='Projection type:',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    style = {'button_width':'200px'}, layout={'width': '520px'},
)

apply_changes_widget = widgets.Button(
    description='Apply changes',
    tooltip='Apply changes',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    style = style,
)
period_widget = widgets.IntRangeSlider(
    min=0,
    max=168,
    value=[0, 168],
    step=1,
    description="Time range in hours (max 1 week from the first event)",
    disabled=False,
    style = style,
    layout={'width': '520px'}
)

ui_widgets = [top_n_widget, data_type_widget, 
              period_widget, apply_changes_widget,
              ]

def display_widgets():
    for ui_widget in ui_widgets:
        display(ui_widget)

def get_first_time_flow_string():
    return download_data[TIMESTAMP_START].min()

def get_last_time_flow_string():
    return download_data[TIMESTAMP_END].max()

def get_first_time_flow():
    return datetime.datetime.fromisoformat(get_first_time_flow_string())

def get_last_time_flow():
    return datetime.datetime.fromisoformat(get_last_time_flow_string())

def to_datetime(x):
    return datetime.datetime.fromisoformat(x)

def process_by_top_n(current_data, top_n):
    if current_data.empty:
        return current_data
    return current_data.nlargest(top_n, columns=DATA_AMOUNT)

def process_by_time_range(current_data, time_period):
    if current_data.empty:
        return current_data
    
    start_offset, end_offset = time_period


    start = get_first_time_flow() + datetime.timedelta(hours=start_offset)
    end = get_first_time_flow() + datetime.timedelta(hours=end_offset)

    return current_data[(current_data[TIMESTAMP_START].apply(to_datetime) >= start) &
                      ((current_data[TIMESTAMP_END].apply(to_datetime)) <= end)]

#downloaded data amount is not multiplied by Download count
def process_data_download(current_data: pd.DataFrame, top_n, time_period):
    processed_data = process_by_time_range(current_data, time_period)
    processed_data = processed_data.groupby(FILE_NAME, as_index=False).aggregate({DATA_AMOUNT: 'sum', DOWNLOAD_COUNT: 'sum', 
                                                                                  FILE_EXT: 'first', APP: 'first'})
    return process_by_top_n(processed_data, top_n)

def make_download_plot(current_data):
    plot = alt.Chart(current_data).mark_bar().encode(
        x=alt.X(DATA_AMOUNT),
        y=alt.Y(FILE_NAME, sort='-x'),
        color=alt.Color(DATA_AMOUNT,
                   scale=alt.Scale(range=['lightgreen', 'green'])),
         tooltip=[
                 alt.Tooltip(f"{FILE_NAME}", title="File name"),
                 alt.Tooltip(f"{FILE_EXT}", title="File extension"),
                 alt.Tooltip(f"{APP}", title="Download via"),
                 alt.Tooltip(f"{DATA_AMOUNT}:Q", title="Downloaded data amout"),
                 alt.Tooltip(f"{DOWNLOAD_COUNT}:Q", title="Download count"),
             ],
    ).properties(
        title=f"File size download",    
        width=800, 
        height=500,
    )
    return alt.JupyterChart(plot)
    
    
def on_change(v):
    clear_output(wait=True)
    top_n = int(top_n_widget.value)
    display_widgets()

    if data_type_widget.value == DATA_AMOUNT_DSC:
        processed_data = process_data_download(download_data, top_n, period_widget.value)
    
    if processed_data.empty:
        print("Log: The data set is empty")
        return
    
    if (data_type_widget.value == DATA_AMOUNT_DSC):
        plot: alt.JupyterChart = make_download_plot(processed_data)

    
    display(plot)


display_widgets()
apply_changes_widget.on_click(on_change)

Dropdown(description='Top N stats', index=4, options=('10', '20', '30', '40', '50'), style=DescriptionStyle(de…

ToggleButtons(button_style='info', description='Projection type:', layout=Layout(width='520px'), options=('Dow…

IntRangeSlider(value=(0, 168), description='Time range in hours (max 1 week from the first event)', layout=Lay…

Button(button_style='success', description='Apply changes', style=ButtonStyle(), tooltip='Apply changes')

JupyterChart(spec={'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300}}, 'data': {'name': 'da…